In [1]:
import numpy as np
from pathlib import Path
import sys
sys.path.append(str(Path().resolve().parent))
import PixelArrays as array

In [2]:
# Total severity score in %
total_severity_score = {
    0: (0,0),
    1: (0, 25), 
    2: (26, 50), 
    3: (51, 75),
    4: (76, 100)
}

def calculate_ratio(lung_mask, infection_mask):
    lung_area = np.count_nonzero(lung_mask)
    infection_area = np.count_nonzero(infection_mask)
    ratio = infection_area / lung_area * 100
    tts = 0
    for score, (lower, upper) in total_severity_score:
        if ratio >= lower & ratio <= upper:
            tts = score
            break
    
    return tts

In [ ]:
def tests_segmented_nii():
    path = Path(r"D:\Studia\sem7\inzynierka\data\segmented_nii_kaggle")
    lungs_path = path / "lung_mask"
    infection_path = path / "infection_mask"
    lungs_files = [x for x in lungs_path.glob('**/*') if x.is_file()]
    infection_files = [x for x in infection_path.glob('**/*') if x.is_file()]
    lungs_masks = [array.get_pixel_array_nii_all(x) for x in lungs_files]
    return lungs_masks
    
l = tests_segmented_nii()

In [12]:
l

[memmap([[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
 
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
 
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
 
         ...,
 
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0.